# Sistema de agrupación de casas (California Housing)
Este notebook implementa un flujo **no supervisado → supervisado**:
1) **K-Means** para agrupar casas usando **Latitude, Longitude y MedInc**.
2) Se asigna el **cluster** a cada punto.
3) Se entrena un **clasificador supervisado** para predecir el cluster.
4) Se guardan los modelos.

> Nota: Aunque K-Means es no supervisado, aquí se usa `train/test` para entrenar en train y luego **predecir clusters** en test, tal como pide la guía.

## 1. Imports y configuración

In [ ]:
# Se importan librerías base de análisis y visualización.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Se importan utilidades de ML para dividir datos, escalar y entrenar modelos.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Se importan métricas y un modelo supervisado para aprender los clusters.
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Se importa joblib para guardar modelos (más cómodo que pickle para sklearn).
import joblib

# (Opcional) Para que los resultados de numpy/pandas sean más legibles.
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)


## 2. Carga del dataset
Solo se usarán: **Latitude**, **Longitude**, **MedInc**.

**Tip:** si el enlace falla, se puede descargar el CSV y cambiar `DATA_PATH` a la ruta local.

In [ ]:
# Se define la ruta del dataset (URL o ruta local).
DATA_PATH = "https://breathecode.herokuapp.com/asset/internal-link?id=809&path=housing.csv"

# Se carga el dataset.
df = pd.read_csv(DATA_PATH)

# Se revisa la estructura.
print("Shape:", df.shape)
display(df.head())

# Se revisa información general (tipos y nulos).
display(df.info())

# Se seleccionan únicamente las columnas solicitadas.
cols = ["Latitude", "Longitude", "MedInc"]
df = df[cols].copy()

# Se revisa el dataset ya filtrado.
print("\nColumnas usadas:", df.columns.tolist())
display(df.describe())


## 3. Limpieza rápida
K-Means no acepta nulos. Si hay nulos, se imputan o se eliminan filas.
Aquí se elimina cualquier fila con nulos en las 3 columnas (suficiente para el ejercicio).

In [ ]:
# Se cuentan nulos por columna.
print("Nulos por columna:\n", df.isnull().sum())

# Se eliminan filas con nulos en las columnas usadas.
df = df.dropna().reset_index(drop=True)

print("\nShape después de dropna:", df.shape)


## 4. Split train/test (80/20)
Aunque es no supervisado, se hace split para entrenar en train y luego predecir clusters en test.

In [ ]:
# Se separa X con las 3 features.
X = df[["Latitude", "Longitude", "MedInc"]].copy()

# Se hace split 80/20 con semilla para replicabilidad.
X_train, X_test = train_test_split(
    X,
    test_size=0.2,
    random_state=42
)

print("Train:", X_train.shape, "| Test:", X_test.shape)


## 5. Escalado (muy importante en K-Means)
K-Means usa distancias: si no se escala, la variable con mayor rango domina.
Se usa **StandardScaler** (media 0, desviación 1).

In [ ]:
# Se inicializa el escalador.
scaler = StandardScaler()

# Se ajusta en train y se transforma train/test.
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Se verifica (opcional) que el escalado se aplicó.
print("Media aproximada (train):", X_train_scaled.mean(axis=0))
print("Std aproximada (train):", X_train_scaled.std(axis=0))


## 6. K-Means con 6 clusters
Se entrena con `n_clusters=6` y se predice cluster para train y test.

**Nota:** `n_init` se fija para estabilidad (reintentos de inicialización).

In [ ]:
# Se define el modelo K-Means con 6 clusters.
kmeans = KMeans(
    n_clusters=6,     # clusters solicitados
    random_state=42,  # replicabilidad
    n_init=10         # varias inicializaciones para evitar soluciones malas
)

# Se entrena el modelo SOLO con train.
kmeans.fit(X_train_scaled)

# Se obtienen clusters para train y test.
train_clusters = kmeans.predict(X_train_scaled)
test_clusters = kmeans.predict(X_test_scaled)

# Se crean copias para no modificar X_train/X_test originales.
train_df = X_train.copy()
test_df = X_test.copy()

# Se agrega la columna de cluster.
train_df["cluster"] = train_clusters
test_df["cluster"] = test_clusters

# Se revisa distribución de clusters en train/test.
print("Distribución clusters (train):\n", train_df["cluster"].value_counts().sort_index())
print("\nDistribución clusters (test):\n", test_df["cluster"].value_counts().sort_index())

display(train_df.head())


## 7. Visualización de clusters (scatter)
Se grafica **Longitude vs Latitude** y el color representa el cluster.
Se dibuja train y luego se sobreponen los puntos de test con marcador diferente.

In [ ]:
# Se crea una figura.
plt.figure(figsize=(10, 6))

# Se grafica TRAIN.
plt.scatter(
    train_df["Longitude"], train_df["Latitude"],
    c=train_df["cluster"],
    cmap="tab10",
    s=12,
    alpha=0.6,
    label="Train"
)

# Se grafica TEST encima.
plt.scatter(
    test_df["Longitude"], test_df["Latitude"],
    c=test_df["cluster"],
    cmap="tab10",
    s=18,
    marker="x",
    alpha=0.9,
    label="Test"
)

# Se configuran títulos y ejes.
plt.title("K-Means (k=6) - Clusters por ubicación (Train vs Test)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.tight_layout()
plt.show()


### ¿Qué se debería observar?
- Zonas geográficas con colores similares indican **agrupamientos espaciales**.
- Como se incluye `MedInc`, clusters pueden separar regiones que comparten ubicación pero difieren en ingreso medio.
- Los puntos de test deberían caer en regiones coherentes con los clusters de train (misma estructura).

## 8. Modelo supervisado para predecir clusters
Ahora el **cluster** se usa como etiqueta (`y`) para entrenar un clasificador.
Se elige **RandomForestClassifier** porque:
- funciona bien sin supuestos fuertes,
- captura no linealidades,
- no requiere normalidad.

El objetivo aquí NO es predecir una variable real, sino aprender a reproducir la partición generada por K-Means.

In [ ]:
# Se definen X e y para clasificación usando los datos escalados.
X_train_sup = X_train_scaled
y_train_sup = train_clusters

X_test_sup = X_test_scaled
y_test_sup = test_clusters

# Se inicializa el modelo supervisado.
clf = RandomForestClassifier(
    n_estimators=300,   # número de árboles
    random_state=42,    # replicabilidad
    n_jobs=-1           # usar todos los cores disponibles
)

# Se entrena el clasificador.
clf.fit(X_train_sup, y_train_sup)

# Se predice en test.
y_pred_sup = clf.predict(X_test_sup)

# Se evalúa.
acc = accuracy_score(y_test_sup, y_pred_sup)
print(f"Accuracy (clasificador vs clusters K-Means): {acc:.4f}")

print("\nMatriz de confusión:")
print(confusion_matrix(y_test_sup, y_pred_sup))

print("\nReporte de clasificación:")
print(classification_report(y_test_sup, y_pred_sup))


## 9. Guardado de modelos
Se guardan:
- `scaler` (para transformar nuevos datos igual que train)
- `kmeans` (para asignar clusters)
- `clf` (para predecir cluster de forma supervisada)

Se guardan en una carpeta `models/`.

In [ ]:
from pathlib import Path

# Se crea la carpeta de salida.
models_dir = Path("models")
models_dir.mkdir(exist_ok=True)

# Se guardan objetos.
joblib.dump(scaler, models_dir / "scaler_standard.pkl")
joblib.dump(kmeans, models_dir / "kmeans_k6.pkl")
joblib.dump(clf, models_dir / "rf_classifier_clusters.pkl")

print("Modelos guardados en:", models_dir.resolve())


## 10. (Opcional) Ejemplo de predicción con puntos nuevos
Este ejemplo muestra cómo usar `scaler` + `kmeans` para asignar cluster a casas nuevas.

In [ ]:
# Se definen algunos puntos nuevos de ejemplo (Latitude, Longitude, MedInc).
new_points = pd.DataFrame({
    "Latitude": [34.05, 37.77, 32.71],
    "Longitude": [-118.24, -122.42, -117.16],
    "MedInc": [4.0, 8.5, 3.2]
})

# Se escala con el MISMO scaler.
new_points_scaled = scaler.transform(new_points)

# Se predice el cluster con K-Means.
new_clusters = kmeans.predict(new_points_scaled)

# Se muestra el resultado.
new_points["cluster_predicho"] = new_clusters
display(new_points)
